# Table of Contents
 <p>

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Font which got unicode math stuff.
import matplotlib as mpl
mpl.rcParams['font.family'] = 'DejaVu Sans'

# Much more readable plots
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Much better than plt.subplots() 
from mpl_toolkits.axes_grid1 import ImageGrid

In [2]:
import functools
import os
import json
import math
import numpy as np
import pickle

import hyperopt as hp
import hyperopt.mongoexp

import objective_functions as ofn

In [3]:
votes_to_detections_space = {
    'bin_size': hyperopt.hp.uniform('bin_size', 0.1, 1.0),
    'vote_collect_radius': hyperopt.hp.uniform('vote_collect_radius', 0.01, 2.0),
    'min_thresh': hyperopt.hp.loguniform('min_thresh', -7*np.log(10), -2*np.log(10)),
    'blur_sigma': hyperopt.hp.uniform('blur_sigma', 0.0, 5.0),
}

In [4]:
run_name = 'DR-SPAAM_11_5_new'
mongodb_port = 27012
trials = hp.mongoexp.MongoTrials('mongo://localhost:{}/hyperopt/jobs'.format(mongodb_port), exp_key=run_name)

In [5]:
inference_result = '/home/jia/v3/hyperopt/inference_result_new.pkl'
with open(inference_result, 'rb') as f:
    scans, pred_cls, pred_reg, gts_xy, gts_inds = pickle.load(f)

In [7]:
votes_to_detections_func = functools.partial(ofn.objective,
                                             scans=scans, 
                                             pred_cls=pred_cls, 
                                             pred_reg=pred_reg,
                                             gts_xy=gts_xy, 
                                             gts_inds=gts_inds)

    
best = hp.fmin(votes_to_detections_func, 
               space=votes_to_detections_space, 
               trials=trials,
               algo=hp.tpe.suggest,
               max_evals=30000)

over-writing old domain trials attachment


 52%|█████▏    | 15491/30000 [17:41:52<16:34:33,  4.11s/trial, best loss: -0.5389506816864014]


KeyboardInterrupt: 

In [9]:
trials.best_trial.values()

[ObjectId('5e57d416d7c82d8659e556c6'),
 2,
 182811,
 None,
 SON([('loss', -0.5389506816864014), ('status', 'ok'), ('real_attachments', SON([('kw', b'{"bin_size": 0.10048541940486004, "blur_sigma": 1.459561417325547, "min_thresh": 9.447764939669593e-05, "vote_collect_radius": 0.15719563974052672}'), ('auc', b'0.5389506816864014')]))]),
 SON([('tid', 182811), ('cmd', ['domain_attachment', 'FMinIter_Domain']), ('workdir', None), ('idxs', SON([('bin_size', [182811]), ('blur_sigma', [182811]), ('min_thresh', [182811]), ('vote_collect_radius', [182811])])), ('vals', SON([('bin_size', [0.10048541940486004]), ('blur_sigma', [1.459561417325547]), ('min_thresh', [9.447764939669593e-05]), ('vote_collect_radius', [0.15719563974052672])]))]),
 'DR-SPAAM_11_5_new',
 ['ncm0239.hpc.itc.rwth-aachen.de:59526'],
 3,
 datetime.datetime(2020, 2, 27, 14, 37, 11, 293000),
 datetime.datetime(2020, 2, 27, 14, 37, 30, 169000)]

In [23]:
votes_to_detections_func = functools.partial(ofn.objective,
                                             scans=scans, 
                                             pred_cls=pred_cls, 
                                             pred_reg=pred_reg,
                                             gts_xy=gts_xy, 
                                             gts_inds=gts_inds)

votes_to_detections_func({'bin_size': 0.1315193551894875,
                         'blur_sigma': 0.8769606532708437,
                          'min_thresh': 9.436743980879768e-5,
                          'vote_collect_radius': 0.16901292463464487})

{'loss': -0.529565155506134,
 'status': 'ok',
 'real_attachments': {'kw': b'{"bin_size": 0.1315193551894875, "blur_sigma": 0.8769606532708437, "min_thresh": 9.436743980879768e-05, "vote_collect_radius": 0.16901292463464487}',
  'auc': b'0.529565155506134'}}

In [ ]:
hp_values = [(t['result']['loss'], t['misc']['vals']) for t in trials.trials if 'loss' in t['result']]
scores = np.asarray([-t['result']['loss'] for t in trials.trials if 'loss' in t['result']])
keys = hp_values[0][1].keys()
val_count = len(keys)

min_score = np.min(np.exp(scores))
max_score = np.max(np.exp(scores))
norm = mpl.colors.Normalize(min_score, max_score)


fig, ax = plt.subplots(val_count,1,figsize=(18,5*val_count))

for a, k in zip(ax, sorted(keys)):
    hp_vals = np.asarray([h[1][k][0] for h in hp_values])
    if k == 'min_thresh':
        hp_vals = np.log10(hp_vals)
    N, bins, patches = a.hist(hp_vals, bins=100)
    scores_sorted = scores[np.argsort(hp_vals)]
    
    start_idx = 0
    bin_scores = []
    for n in N:
        bin_scores.append(np.mean(scores_sorted[start_idx:start_idx+int(n)]))
        start_idx +=int(n)
    
    for b, thispatch in zip(bin_scores, patches):
        color = plt.cm.viridis(norm(np.exp(b)))
        thispatch.set_facecolor(color)
    a.set_title(k)